**Parsing Notes for "Mayor's Office of Contract Services"::"Meeting"**

These type of messages demonstrates the difficulty in parsing 
unstructured document and strengthens the case for injection
structure/meta at the point of data-entry. 

In [ ]:
from bs4 import BeautifulSoup as Soup
from IPython.display import display, HTML
import pandas as pd
import re
import pprint
import json

In [ ]:
fn = 'orig.procPublicationRequest.oct-dec-2014.csv'
rows = pd.read_csv(fn, header=0, sep='|', encoding='latin-1')
cols = ['RequestID', 'StartDate', 
        'EndDate', 'AgencyCode', 'AgencyName', 
        'AgencyDivision', 'TypeOfNoticeCode', 
        'TypeOfNoticeDescription', 'ShortTitle', 
        'SectionID', 'SectionName', 'DueDate', 
        'ConfirmationNumber', 'AdditionalDescription']
rows = rows[cols]

In [ ]:
def scrape(row):
    output = {}
    try:
        output['text'] = Soup(row.AdditionalDescription).get_text(strip=True)
    except Exception:
        output = { 'error' : 'bad input', 'text': text }
    row['output'] = json.dumps(output)
    return row

In [ ]:
mocs = rows['AgencyName'] == "Mayor's Office of Contract Services"
meets = rows['TypeOfNoticeDescription'] == "Meeting"
fix = rows[mocs & meets].apply(scrape,1)

In [ ]:
for rec in fix[['output', 'AdditionalDescription']].values:
    output, text = rec
    if not isinstance(text, str):
        text = 'NAN'
    display(HTML(text))
    print('---')
    pprint.pprint(json.loads(output))
    display(HTML('<hr />'))
    print('\n\n')